In [21]:
%load_ext autoreload
%autoreload 2
%aimport utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
l1 = [['one', 'two'], ['three', 'four'], ['i', 'love', 'nlp']]
pad_string = 'test'
#l1[0].extend([pad_string]*6)
l1

[['one', 'two'], ['three', 'four'], ['i', 'love', 'nlp']]

In [3]:
def pad_sents(sents, pad_token):
    """ Pad list of sentences according to the longest sentence in the batch.
    @param sents (list[list[str]]): list of sentences, where each sentence
                                    is represented as a list of words
    @param pad_token (str): padding token
    @returns sents_padded (list[list[str]]): list of sentences where sentences shorter
        than the max length sentence are padded out with the pad_token, such that
        each sentences in the batch now has equal length.
    """
    sents_padded = []

    ### YOUR CODE HERE (~6 Lines)
    longest = max([len(sentence) for sentence in sents])
    for idx, sentence in enumerate(sents):
        add = longest - len(sentence)
        sents_padded.append(list(sentence))
        sents_padded[idx].extend([pad_token]*add)
        
    ### END YOUR CODE

    return sents_padded

In [4]:
pad_sents(l1, '<pad>')

[['one', 'two', '<pad>'], ['three', 'four', '<pad>'], ['i', 'love', 'nlp']]

In [5]:
l1

[['one', 'two'], ['three', 'four'], ['i', 'love', 'nlp']]

In [6]:
%whos

Variable     Type        Data/Info
----------------------------------
l1           list        n=3
pad_sents    function    <function pad_sents at 0x7fb71c58bd08>
pad_string   str         test
utils        module      <module 'utils' from '/ho<...>cs224n_2019/a4/utils.py'>


In [12]:
!conda install docopt

^C


In [61]:
!python sanity_check.py 1d

Running Sanity Check for Question 1d: Encode
--------------------------------------------------------------------------------
Traceback (most recent call last):
  File "sanity_check.py", line 235, in <module>
    main()
  File "sanity_check.py", line 224, in main
    question_1d_sanity_check(model, src_sents, tgt_sents, vocab)
  File "sanity_check.py", line 105, in question_1d_sanity_check
    enc_hiddens_pred, dec_init_state_pred = model.encode(source_padded, source_lengths)
  File "/home/jupyter/cs224n_2019/a4/nmt_model.py", line 185, in encode
    enc_hiddens = enc_hiddens.permute(1,0,2)
AttributeError: 'tuple' object has no attribute 'permute'


In [2]:
import torch
from torch import nn

In [3]:
embedding = nn.Embedding(10, 3)
# a batch of 2 samples of 4 indices each
input = torch.LongTensor([[1,2,4,5],[4,3,2,9]])
print(input.size())
embedding(input)

torch.Size([2, 4])


tensor([[[-0.5552,  1.6441, -0.0249],
         [ 1.1960, -0.7706,  1.3190],
         [-0.8076,  0.3844,  0.3498],
         [ 0.7307,  1.0069, -2.6348]],

        [[-0.8076,  0.3844,  0.3498],
         [ 0.2159, -0.0346,  0.4370],
         [ 1.1960, -0.7706,  1.3190],
         [ 0.0715, -2.0978,  1.0670]]], grad_fn=<EmbeddingBackward>)

In [30]:
input.transpose(1,0)

tensor([[1, 4],
        [2, 3],
        [4, 2],
        [5, 9]])

In [6]:
embedding(input).size() #(batch, sentence_length, embedding_dim)

torch.Size([2, 4, 3])

In [34]:
input.transpose(1,0)

tensor([[1, 4],
        [2, 3],
        [4, 2],
        [5, 9]])

In [35]:
input.transpose(1,0).size()

torch.Size([4, 2])

In [33]:
embedding(input.transpose(1,0))

tensor([[[ 0.9403, -1.1993, -0.3030],
         [ 1.4870,  1.9375, -0.5224]],

        [[ 2.3968, -0.5955,  1.1449],
         [ 0.4728,  0.4056, -0.1041]],

        [[ 1.4870,  1.9375, -0.5224],
         [ 2.3968, -0.5955,  1.1449]],

        [[ 1.4181,  0.5341,  0.7078],
         [-0.9772, -0.3458, -0.0610]]], grad_fn=<EmbeddingBackward>)

In [32]:
embedding(input.transpose(1,0)).size()

torch.Size([4, 2, 3])

In [26]:
embedding = nn.Embedding(10, 3)
# a batch of 2 samples of 4 indices each
input = torch.LongTensor([[1,2,4,5]])
print(input.size())
embedding(input)

torch.Size([1, 4])


tensor([[[-0.1033,  1.3244,  0.2117],
         [-0.8510,  1.9430, -1.6529],
         [ 0.1461, -0.2864,  0.4483],
         [ 0.8832,  1.0036,  0.2195]]], grad_fn=<EmbeddingBackward>)

In [27]:
embedding(input).size() #(batch, sentence_length, embedding_dim)

torch.Size([1, 4, 3])

In [41]:
embedding(input.transpose(1,0)).size() # seq_length, batch_size, embedding_dim

torch.Size([4, 2, 3])

In [51]:
packed_embeddings = nn.utils.rnn.pack_padded_sequence(embedding(input.transpose(1,0)), [3, 3])
packed_embeddings

PackedSequence(data=tensor([[ 0.9403, -1.1993, -0.3030],
        [ 1.4870,  1.9375, -0.5224],
        [ 2.3968, -0.5955,  1.1449],
        [ 0.4728,  0.4056, -0.1041],
        [ 1.4870,  1.9375, -0.5224],
        [ 2.3968, -0.5955,  1.1449]], grad_fn=<PackPaddedSequenceBackward>), batch_sizes=tensor([2, 2, 2]), sorted_indices=None, unsorted_indices=None)

In [53]:
torch.nn.utils.rnn.pad_packed_sequence(packed_embeddings, total_length = 4)

(tensor([[[ 0.9403, -1.1993, -0.3030],
          [ 1.4870,  1.9375, -0.5224]],
 
         [[ 2.3968, -0.5955,  1.1449],
          [ 0.4728,  0.4056, -0.1041]],
 
         [[ 1.4870,  1.9375, -0.5224],
          [ 2.3968, -0.5955,  1.1449]],
 
         [[ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000]]], grad_fn=<CopySlices>), tensor([3, 3]))

In [46]:
packed_embeddings = nn.utils.rnn.pack_padded_sequence(embedding(input.transpose(1,0)), [4, 3])
packed_embeddings

PackedSequence(data=tensor([[ 0.9403, -1.1993, -0.3030],
        [ 1.4870,  1.9375, -0.5224],
        [ 2.3968, -0.5955,  1.1449],
        [ 0.4728,  0.4056, -0.1041],
        [ 1.4870,  1.9375, -0.5224],
        [ 2.3968, -0.5955,  1.1449],
        [ 1.4181,  0.5341,  0.7078]], grad_fn=<PackPaddedSequenceBackward>), batch_sizes=tensor([2, 2, 2, 1]), sorted_indices=None, unsorted_indices=None)

In [48]:
torch.nn.utils.rnn.pad_packed_sequence(packed_embeddings)

(tensor([[[ 0.9403, -1.1993, -0.3030],
          [ 1.4870,  1.9375, -0.5224]],
 
         [[ 2.3968, -0.5955,  1.1449],
          [ 0.4728,  0.4056, -0.1041]],
 
         [[ 1.4870,  1.9375, -0.5224],
          [ 2.3968, -0.5955,  1.1449]],
 
         [[ 1.4181,  0.5341,  0.7078],
          [ 0.0000,  0.0000,  0.0000]]], grad_fn=<CopySlices>), tensor([4, 3]))

In [57]:
packed_embeddings = nn.utils.rnn.pack_padded_sequence(embedding(input), [3, 3], batch_first=True)
packed_embeddings

PackedSequence(data=tensor([[ 0.9403, -1.1993, -0.3030],
        [ 1.4870,  1.9375, -0.5224],
        [ 2.3968, -0.5955,  1.1449],
        [ 0.4728,  0.4056, -0.1041],
        [ 1.4870,  1.9375, -0.5224],
        [ 2.3968, -0.5955,  1.1449]], grad_fn=<PackPaddedSequenceBackward>), batch_sizes=tensor([2, 2, 2]), sorted_indices=None, unsorted_indices=None)

In [60]:
torch.nn.utils.rnn.pad_packed_sequence(packed_embeddings, total_length = 4, batch_first = True)

(tensor([[[ 0.9403, -1.1993, -0.3030],
          [ 2.3968, -0.5955,  1.1449],
          [ 1.4870,  1.9375, -0.5224],
          [ 0.0000,  0.0000,  0.0000]],
 
         [[ 1.4870,  1.9375, -0.5224],
          [ 0.4728,  0.4056, -0.1041],
          [ 2.3968, -0.5955,  1.1449],
          [ 0.0000,  0.0000,  0.0000]]], grad_fn=<TransposeBackward0>),
 tensor([3, 3]))

In [65]:
torch.nn.utils.rnn.pad_packed_sequence(packed_embeddings, total_length = 4, batch_first = True)[0]

tensor([[[ 0.9403, -1.1993, -0.3030],
         [ 2.3968, -0.5955,  1.1449],
         [ 1.4870,  1.9375, -0.5224],
         [ 0.0000,  0.0000,  0.0000]],

        [[ 1.4870,  1.9375, -0.5224],
         [ 0.4728,  0.4056, -0.1041],
         [ 2.3968, -0.5955,  1.1449],
         [ 0.0000,  0.0000,  0.0000]]], grad_fn=<TransposeBackward0>)

In [3]:
!python sanity_check.py 1d

Running Sanity Check for Question 1d: Encode
--------------------------------------------------------------------------------
enc_hiddens Sanity Checks Passed!
dec_init_state[0] Sanity Checks Passed!
dec_init_state[1] Sanity Checks Passed!
--------------------------------------------------------------------------------
All Sanity Checks Passed for Question 1d: Encode!
--------------------------------------------------------------------------------


In [74]:
embedding(input).size()

torch.Size([2, 4, 3])

In [4]:
test = embedding(input)
test.size()

torch.Size([2, 4, 3])

In [8]:
#torch.cat((test_concat[0, :, :], test_concat[1, :, :]), dim=1).size()

torch.Size([4, 6])

In [43]:
test

tensor([[[-0.0056, -0.4001,  1.1009],
         [-0.1381,  1.6090, -0.2963],
         [-0.6630,  0.5409, -1.5764],
         [-0.4491, -1.6503, -1.0946]],

        [[-0.6630,  0.5409, -1.5764],
         [-0.4842,  2.0309,  0.4387],
         [-0.1381,  1.6090, -0.2963],
         [ 0.6110, -0.8862,  1.4394]]], grad_fn=<EmbeddingBackward>)

In [44]:
test.size()

torch.Size([2, 4, 3])

In [45]:
torch.split(test, 1, dim=0)

(tensor([[[-0.0056, -0.4001,  1.1009],
          [-0.1381,  1.6090, -0.2963],
          [-0.6630,  0.5409, -1.5764],
          [-0.4491, -1.6503, -1.0946]]], grad_fn=<SplitBackward>),
 tensor([[[-0.6630,  0.5409, -1.5764],
          [-0.4842,  2.0309,  0.4387],
          [-0.1381,  1.6090, -0.2963],
          [ 0.6110, -0.8862,  1.4394]]], grad_fn=<SplitBackward>))

In [47]:
torch.split(test, 1, dim=0)[0]

tensor([[[-0.0056, -0.4001,  1.1009],
         [-0.1381,  1.6090, -0.2963],
         [-0.6630,  0.5409, -1.5764],
         [-0.4491, -1.6503, -1.0946]]], grad_fn=<SplitBackward>)

In [46]:
torch.split(test, 1, dim=0)[0].size()

torch.Size([1, 4, 3])

In [48]:
torch.split(test, 1, dim=0)[0].squeeze()

tensor([[-0.0056, -0.4001,  1.1009],
        [-0.1381,  1.6090, -0.2963],
        [-0.6630,  0.5409, -1.5764],
        [-0.4491, -1.6503, -1.0946]], grad_fn=<SqueezeBackward0>)

In [49]:
torch.split(test, 1, dim=0)[0].squeeze().size()

torch.Size([4, 3])

In [52]:
time_steps = torch.split(test, 1, dim=0)
for t in time_steps:
    print(t.squeeze().size())

torch.Size([4, 3])
torch.Size([4, 3])


In [54]:
test.size()

torch.Size([2, 4, 3])

In [75]:
test[0].size(), test[1].size()

(torch.Size([4, 3]), torch.Size([4, 3]))

In [72]:
test[0, :, :].size(), test[1, :, :].size()

(torch.Size([4, 3]), torch.Size([4, 3]))

In [77]:
torch.cat((test[0, :, :], test[1, :, :])).size()

torch.Size([8, 3])

In [65]:
torch.cat((test[0, :, :], test[1, :, :]), dim=1).size()

torch.Size([4, 6])

In [71]:
torch.cat((test[0,:,:], test[1, :, :]), dim=0).size()

torch.Size([8, 3])

In [69]:
test.size()

torch.Size([2, 4, 3])

In [ ]:
test

In [67]:
l = []
for t in test:
    l.append(t)
l    

[tensor([[-0.0056, -0.4001,  1.1009],
         [-0.1381,  1.6090, -0.2963],
         [-0.6630,  0.5409, -1.5764],
         [-0.4491, -1.6503, -1.0946]], grad_fn=<SelectBackward>),
 tensor([[-0.6630,  0.5409, -1.5764],
         [-0.4842,  2.0309,  0.4387],
         [-0.1381,  1.6090, -0.2963],
         [ 0.6110, -0.8862,  1.4394]], grad_fn=<SelectBackward>)]

In [70]:
torch.stack(l).size()

torch.Size([2, 4, 3])

In [79]:
!python sanity_check.py 1e

--------------------------------------------------------------------------------
Running Sanity Check for Question 1e: Decode
--------------------------------------------------------------------------------
combined_outputs Sanity Checks Passed!
--------------------------------------------------------------------------------
All Sanity Checks Passed for Question 1e: Decode!
--------------------------------------------------------------------------------


In [6]:
test.size()

torch.Size([2, 4, 3])

In [31]:
test.unsqueeze(dim=1).size()

torch.Size([2, 1, 4, 3])

In [16]:
test[0].unsqueeze(dim=-1).size()

torch.Size([4, 3, 1])

In [17]:
test_broadcast = test[:,:, 0].unsqueeze(dim=-1)
test_broadcast.size()

torch.Size([2, 4, 1])

In [18]:
test.size()

torch.Size([2, 4, 3])

In [ ]:
torch.bmm()

In [ ]:
torch.nn.functional.tanh

In [46]:
!python sanity_check.py 1f

--------------------------------------------------------------------------------
Running Sanity Check for Question 1f: Step
--------------------------------------------------------------------------------
dec_state[0] Sanity Checks Passed!
dec_state[1] Sanity Checks Passed!
combined_output  Sanity Checks Passed!
e_t Sanity Checks Passed!
--------------------------------------------------------------------------------
All Sanity Checks Passed for Question 1f: Step!
--------------------------------------------------------------------------------
